In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked
from utils.metrics import time_span_metrics
from utils.preserves import load_object
from utils.watchmen import LimitWatchman, LimitPcaWatchman, SpePcaWatchman, IsolatingWatchman, LinearPredictWatchman

In [4]:
SEED = 127

# Preparing datasets

In [5]:
datasets = {
    0: GhlKasperskyDataset(),
    1: TepHarvardDataset(),
    2: TepKasperskyDataset(),
    3: SwatItrustDataset(),
}

In [6]:
shake_kwargs = {
    'random_state': SEED,
    'valid_test_ratio': 0.0,
}
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)

# Preparing watchmen

In [7]:
watchmen = {
    0: {w: None for w in range(5)},
    1: {w: None for w in range(5)},
    2: {w: None for w in range(5)},
    3: {w: None for w in range(5)},
}

In [8]:
for d in datasets:
    for w in watchmen[d]:
        watchmen[d][w] = load_object(f'{datasets[d].__class__.__name__}-watchman_{w}')

# Examine

## Throw stones

In [9]:
stones = dict()
for d in datasets:
    stones[d] = dict()
    for w in watchmen[d]:
        stones[d][w] = list()

In [10]:
for d in datasets:
    for data, faults, info in tqdm(datasets[d].test_generator(), desc=f'Detect on test {d}'):
        for w in watchmen[d]:
            detect = watchmen[d][w].predict(data)
            stones[d][w].append(pd.concat([faults, detect], axis=1))
            stones[d][w][-1].index.name = info

Detect on test 0: 0it [00:00, ?it/s]

Detect on test 1: 0it [00:00, ?it/s]

Detect on test 2: 0it [00:00, ?it/s]

Detect on test 3: 0it [00:00, ?it/s]

## Results

In [11]:
metrics = ('precision', 'recall', 'f1_score')
results = {str(datasets[d]): pd.DataFrame(columns=metrics) for d in datasets}

### Individual

In [12]:
for d in datasets:
    for w in tqdm(watchmen[d], desc=f'Collect stones d{d}'):
        exam_paper = pd.DataFrame(columns=metrics)
        for i_st, st in enumerate(stones[d][w]):
            exam_paper.loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], st.iloc[:, 1:])
        results[str(datasets[d])].loc[str(watchmen[d][w]), metrics] = exam_paper.mean().values

Collect stones d0:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d1:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d2:   0%|          | 0/5 [00:00<?, ?it/s]

Collect stones d3:   0%|          | 0/5 [00:00<?, ?it/s]

### Ensembling

In [13]:
threshold = 4
for d in datasets:
    exam_paper = pd.DataFrame(columns=metrics)
    for i_st, st in tqdm(enumerate(stones[d][0]), desc=f'Collect stones d{d}'):
        opinions = pd.concat([stones[d][w][i_st].iloc[:, 1:].sum(axis=1) for w in watchmen[d]], axis=1)
        detect = (opinions.sum(axis=1) >= threshold).astype('uint8')
        exam_paper.loc[i_st, metrics] = time_span_metrics(st.iloc[:, 0], detect)
    results[str(datasets[d])].loc[f'ensemble(threshold={threshold})', metrics] = exam_paper.mean().values

Collect stones d0: 0it [00:00, ?it/s]

Collect stones d1: 0it [00:00, ?it/s]

Collect stones d2: 0it [00:00, ?it/s]

Collect stones d3: 0it [00:00, ?it/s]

### Let's see

In [17]:
for d in results:
    print(d)
    display(results[d].sort_values(by='f1_score', ascending=False))

GhlKasperskyDataset(E:\Datasets\GHL)


,precision,recall,f1_score
LimitWatchman(ewma=None),0.32572,0.643899,0.378634
LimitPcaWatchman(n_components=3),0.369174,0.383978,0.355303
ensemble(threshold=4),0.348501,0.315377,0.300894
IsolatingWatchman(n_trees=17),0.016294,0.970982,0.031934
LinearPredictWatchman(n_features=12),0.013827,1.0,0.027189
SpePcaWatchman(n_components=3),0.0,0.0,0.0


TepHarvardDataset(E:\Datasets\TEP\dataverse)


,precision,recall,f1_score
LinearPredictWatchman(n_features=52),0.983441,0.748545,0.771986
LimitWatchman(ewma=None),0.990542,0.640096,0.679066
ensemble(threshold=4),0.999266,0.591363,0.641562
SpePcaWatchman(n_components=3),0.999744,0.548044,0.588005
IsolatingWatchman(n_trees=500),0.854073,0.454568,0.499293
LimitPcaWatchman(n_components=3),0.996004,0.402341,0.449245


TepKasperskyDataset(E:\Datasets\TEP\kaspersky)


,precision,recall,f1_score
LimitWatchman(ewma=None),0.765249,0.433985,0.413122
ensemble(threshold=4),0.839017,0.30976,0.323324
LinearPredictWatchman(n_features=53),0.359084,0.478348,0.255118
SpePcaWatchman(n_components=3),0.976871,0.245566,0.253022
LimitPcaWatchman(n_components=3),0.836735,0.243649,0.240436
IsolatingWatchman(n_trees=400),0.075873,1.0,0.116781


SwatItrustDataset(E:\Datasets\SWaT\dataset12)


,precision,recall,f1_score
ensemble(threshold=4),0.347633,0.783333,0.348253
LimitPcaWatchman(n_components=3),0.341508,0.432456,0.344494
LinearPredictWatchman(n_features=51),0.338323,0.671345,0.323797
LimitWatchman(ewma=None),0.193875,0.983333,0.309055
IsolatingWatchman(n_trees=6),0.187796,1.0,0.300011
SpePcaWatchman(n_components=3),1.0,0.033333,0.060606
